In [18]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [19]:
import numpy as np
import scipy
import librosa

from IPython.lib.display import Audio
from lpc import *

**Define some marcos**

In [20]:
ORDER_P = 8
PULSE_FREQ = 100 # will used in one of the following lpc synthesis

## Transmit

In [21]:
d,sr = librosa.load('LPC-test.m4a',sr = 8000)
print("Number of data points: {}".format(d.shape[0]))
[a,g,e] = lpcfit(d,ORDER_P)
print("Number of frames: {}".format(a.shape[0]))

Number of data points: 25934
Number of frames: 202


## Receive

### Perfect uncompressed e[n]

at every timestamp will record its value, useless in practical XD


In [22]:
rx_lossless_d = lpcsynth(a,g,e)
print('Number of recorded data points in excitation if lossless: {}'.format(e.shape[0]))
Audio(rx_lossless_d,rate = sr)

Number of recorded data points in excitation if lossless: 25920


### White noise as e[n]
discard excitation, use white noise directly

In [23]:
rx_noise_d = lpcsynth(a,g)
Audio(rx_noise_d,rate=sr)

從 LPC 的定義可以看出， 理想上{a_k} 要包含所有訊號間 linear dependence 的關係，所以剩下的 excitation 每個時間點的訊號彼此沒有關聯，就像 white noise 一樣，但還是可以聽出聲音有些 **沙啞感**

### Pulse Train as e[n]

In [24]:
rx_pulse_d = lpcsynth(a,g,PULSE_FREQ)
Audio(rx_pulse_d,rate=sr)

機器人音！！ (用固定的 pitch 發聲)

### Buzz-Hiss Coding

#### encode

In [25]:
pitch_e = lpcBHenc(e) 
print('After buzz-hiss encoding, number of recorded data points in excitation: {}'.format(pitch_e.shape[0])) # equals to num_frame

After buzz-hiss encoding, number of recorded data points in excitation: 202


每個 frame 記一個 pitch 的值即可

#### decode

In [26]:
rx_pitch_track_d = lpcsynth(a,g,lpcBHdec(pitch_e))
Audio(rx_pitch_track_d,rate=sr)

可以聽出音質的確比 white noise 好很多！ (但有些地方會暴音)